In [1]:
import re
from collections import Counter
import pandas as pd 
import numpy as np




import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')





import difflib



import math
WORD = re.compile(r"\w+") 




In [2]:
def words(text): return re.findall(r'\w+', text.lower())

In [3]:
data = pd.read_csv("D:/Indian Personality/first10100.csv") 

In [4]:
data_categories = data['Categories']
title = data['Title']
content = data['Content']

In [5]:
"""
Now we'll convert data_categories into proper list to be used 
"""

def to_list(s) :
    s = s.replace("[","")
    s = s.replace("]","")
    s = s.replace("'","")
    s = s.replace("'","")
    
    return list(s.split(","))

In [6]:
%%time 


for i in range(10088) :
    data_categories[i] = to_list(data_categories[i])

<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 1.39 s


In [7]:
import re

# If you need to use the regex more than once it is suggested to compile it.
pattern = re.compile(r"[\n]+")
pattern2 = re.compile(r"[=]+")

In [8]:
%%time
for i in range(10088) :
    content[i] = pattern.sub("",content[i])
    content[i] = pattern2.sub("",content[i])

<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 7.68 s


In [9]:
len(stop)

179

In [10]:
%%time

text = " "
for i in range(10088) :
    text += ''.join(data_categories[i])
    
    
for i in range(10088) :
    text += title[i]
    text += " "
    


Wall time: 9.97 s


In [11]:
%%time
WORDS = Counter(words(text))

Wall time: 635 ms


In [12]:
def correction_inside(word) :
    words = word.split()
    final_suggestion_list = []
    final_suggestion_dict = {}
    final_ans = []
    for w in words :
        w = w.lower()
        cor_word = []
        counter = 0 
        max_score = 0 
        text = " "
        for i in WORDS :
            i = i.lower()
            #print(i,word)
            #print(difflib.SequenceMatcher(None,word,i).ratio()) 
            score = difflib.SequenceMatcher(None,w,i).ratio()
            """
            Apart from sending max word we'll also certain other combinations 
            """
                        
            if(score > 0.7) :
                cor_word.append(i)
                
            
            
            if(score > max_score) :
                max_score = score 
                text = i
        final_ans.append(text)
        final_suggestion_dict[w] = cor_word
                
            

        
            
            
    return final_ans , final_suggestion_dict

In [13]:
def correction(words) :
    words = words.split(" ")
    word = "" 
    for w in words :
        w = w.lower()
        if w not in stop :
            word += w
            word += " "
    #print("Input query  ",word)
    correct_word , suggestions = correction_inside(word) 
    print("Searching for .......    ",correct_word,"\n")
    print("Here is the list of similar words you may be looking for ...............\n")
    for k in suggestions :
        print(k,"    ",suggestions[k],"\n")
    
    
    print("If the suggestion is correct then press continue otherwise you can research your query")
    return correct_word

In [14]:
correction("When was Atal Vihari Vajpayee born")

Searching for .......     ['atal', 'bihari', 'vajpayee', 'born'] 

Here is the list of similar words you may be looking for ...............

atal      ['patiala', 'factual', 'data', 'santali', 'martial', 'tata', 'talk', 'coastal', 'kaithal', 'capital', 'partial', 'natural', 'akal', 'catalan', 'pattali', 'iata', 'natal', 'atal', 'actual', 'rateall', 'artsall', 'atul', 'bantwal', 'bhatkal', 'mata', 'aitwal', 'amal', 'aata', 'atwal', 'paintal', 'bhattal', 'atrauli', 'tala', 'ajitpal', 'ratilal', 'lata', 'katchal', 'mathpal', 'batalvi', 'satpal', 'matilal', 'vatsal', 'vatal'] 

vihari      ['bihar', 'motihari', 'bihari', 'tihar', 'hari', 'vaishali', 'niharika', 'kavishar', 'bhikhari', 'avari', 'virji', 'srihari', 'vinubhai', 'vibha', 'ishar'] 

vajpayee      ['jaypee', 'vajpayee', 'jayee'] 

born      ['korn', 'osborne', 'horn', 'bryn', 'boer', 'bone', 'borneo', 'born', 'bora', 'bond', 'boro'] 

If the suggestion is correct then press continue otherwise you can research your query


['atal', 'bihari', 'vajpayee', 'born']

##### Keywords  

In [15]:
%%time

### Keyword will store all keywords 

keyword = [0 for i in range(10000000)]
counter = 0
for i in range(10088) :
    keyword[counter] = title[i].lower()
    counter += 1 
    
for i in range(10088) :
    for j in range(len(data_categories[i])) :
        keyword[counter] = data_categories[i][j].lower()
        counter += 1 


Wall time: 4.9 s


In [16]:
"When was Atal Vihari Vajpayee born".lower()

'when was atal vihari vajpayee born'

In [17]:
for i in range(counter) :
    if keyword[i] in "When was Atal Bihari Vajpayee born".lower() :
        print(keyword[i])

atal bihari vajpayee
ay
bihar
 bihar
 bihar
 bihar
 atal bihari vajpayee
 atal bihari vajpayee
 bihar
 bihar
 bihar
 bihar
 bihar
 bihar


In [18]:
keyword = keyword[:counter]
keyword = list(set(keyword))
counter = len(keyword)
print(counter)

37199


In [141]:
def find_relevant_page(query) :
    query = query.lower()
    keyword_list = [0 for I in range(10000)]
    counterL = 0 
    for i in range(counter) :
        if keyword[i] in query :
            #print(keyword[i])
            keyword_list[counterL] = keyword[i]
            counterL += 1
    return list(set(keyword_list[:counterL]))

def is_title_imp(query) :
    query = query.lower()
    keyword_list = [0 for I in range(10000)]
    counterL = 0 
    for i in range(10088) :
        if title[i].lower() in query :
            #print(keyword[i])
            for j in range(len(data_categories[i])) :
                if 'births' in data_categories[i][j].lower() :
                    keyword_list[counterL] = title[i]
                    counterL += 1
                    print("Added .......",title[i],"\n")
                    break 
                    
                if 'deaths' in data_categories[i][j].lower() :
                    keyword_list[counterL] = title[i]
                    counterL += 1
                    print("Added .......",title[i],"\n")
                    
    
    return list(set(keyword_list[:counterL]))



def find_keyword(query) :
    List = is_title_imp(query) 
    if( not len(List)) :
        print("No title found .........................\n\n")
        List = find_relevant_page(query)
    #### Removing the keyword "Who" from it 
    if "who" in List :
        List.remove("who")
    return List
            
   

#### Correcting keyword part is yet to be done  

In [20]:
def unique(keywords):
    keys = []
    for s in keywords:
        if not any([s in r for r in keywords if s != r]):
            keys.append(s)
    
    return keys

In [21]:
imp_keywords = unique(find_keyword("list of bharat ratna from maharashtra"))
imp_keywords 

No title found .........................




[' maharashtra', 'bharat ratna']

#### Finding all relevant pages  

In [22]:
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [23]:
%%time
for i in range(10088) :
    ## Converting all cat of this title to lower 
    for j in range(len(data_categories[i])) :
        data_categories[i][j] = data_categories[i][j].lower()

Wall time: 7.42 s


In [80]:
def content_find(keywords) :
    
    print("Case 0\n")
    contentZero = [0 for i in range(10088)]
    contentLent = 0 
    for i in range(10088) :
        if keywords[0].lower() == title[i].lower() :
            contentZero[contentLent] = content[i].lower() 
            contentLent += 1
    if(contentLent) :
        return contentZero[:contentLent] , 1
    
    contents = {}
    content_final = [0 for i in range(1000)]
    final_counter =  0 
    
    
    list_of_title = [0 for i in range(100088)] 
    counterL = 0 
    done = 0 # Done is 1 if we have got titles matching keywords 
    ## First looking for any title matching [But only if number of keywords is 1]
    if(len(keywords) == 1) :
        print("Case 1")
        for i in range(10088) :
            if keywords[0].lower() in title[i].lower() :
                print(title[i].lower()," and index is ",i,"\n")
                done = 1
                contents[get_cosine(text_to_vector(keywords[0]),text_to_vector(title[i]))] = (content[i],title[i])
                    
        
        l = list(contents.items())
        l.sort(reverse=True)
        contents = dict(l)
       # print("Sorted and converted to dictionary again ")
        only5 = 0 
        for k in contents :
            only5+=1 
            content_final[final_counter] = contents[k][0]
            final_counter += 1
            #print("Content and title are  ..........",contents[k][0],"\n\n\n",contents[k][1])
            if(only5 == 5) :
                break 
        if(done == 1 ) :
            return content_final[:final_counter] ,1 
    
    
    ### The case we'll have to return list of answer is left 
    index_title = [0 for i in range(100088)] 
    
    if(done == 0) :
        print("Case 2 ")
        all_titles = [0 for i in range(10088)]
        for i in range(10088) : 
            done = 0 
            lenK = len(keywords)
            for k in range(lenK) :
                for z in range(len(data_categories[i])) :
                    if keywords[k] in data_categories[i][z].lower() :
                        done += 1 
                        break 
            if(done == lenK) :
                list_of_title[counterL] = title[i] 
                index_title[counterL] = i
                counterL += 1 
                print("Title added is ...................",title[i])
        if(counterL < 4) :
            print("Inside here")
            for i in range(4) :
                content_final[final_counter] = content[index_title[i]]
                print(content_final[final_counter])
                final_counter += 1 
            return content_final[:final_counter] , 1
            
            
        return list_of_title[:counterL],2


In [25]:
sample_key = ['kalpana chawla']

content_list , number = content_find(sample_key)

Case 1
kalpana chawla  and index is  9779 



In [26]:
def all_preprocessing(query) :
    query = query.lower()
    ##Spelling correction 
    correct = correction(query) 
    done = int(input()) 
    while(not done) :
        print("Input your query again .......................")
        query = input(query)
        #correct = correction(query)
        done = int(input())
    imp_keywords,num = unique(find_keyword(query))
    #### Removing the extra ' ' from front from all keywords 
    for i in range(len(imp_keywords)) :
        if(imp_keywords[i][0] == ' ') :
            #print(imp_keywords[i])
            imp_keywords[i] = imp_keywords[i][1:] 
    print("imp keyword is ................",imp_keywords)
    content_list , number = content_find(imp_keywords)
    
    return content_list , number 

    

In [27]:
query = "What kalpana chawla spoke"
Con_list , num_type = all_preprocessing(query)

Searching for .......     ['kalpana', 'chawla', 'spoken'] 

Here is the list of similar words you may be looking for ...............

kalpana      ['maulana', 'kala', 'kalakar', 'apna', 'albania', 'nalanda', 'kalyani', 'kalpana', 'kaalapani', 'pallana', 'kalyan', 'kapadia', 'aparna', 'kalpen', 'kalpeni', 'kaliprasanna', 'kalpnath', 'kalanaur', 'kalyana', 'kalipada', 'kalakankar', 'kapany', 'kalanath', 'kangana'] 

chawla      ['chalukya', 'chautala', 'chola', 'hawai', 'chakwal', 'chawla', 'chaliha', 'hala', 'chandela', 'chamkila', 'chandola'] 

spoke      ['spoken', 'spikes', 'smoked', 'poker', 'stokes', 'ashoke'] 

If the suggestion is correct then press continue otherwise you can research your query
1
No title found .........................


imp keyword is ................ ['kalpana chawla']
Case 1
kalpana chawla  and index is  9779 



In [28]:
Con_list

['Kalpana Chawla (17 March 1962 – 1 February 2003) was an American astronaut, engineer, and the first woman of Indian origin to go to space. She first flew on Space Shuttle Columbia in 1997 as a mission specialist and primary robotic arm operator. In 2003, Chawla was one of the seven crew members who died in the Space Shuttle Columbia disaster when the spacecraft disintegrated during its re-entry into the Earth\'s atmosphere. Chawla was posthumously awarded the Congressional Space Medal of Honor, and several streets, universities, and institutions have been named in her honor. She is regarded as a national hero in India. Early life Kalpana Chawla was born on 17 March 1962, in Karnal of present-day Haryana, India, but her official date of birth was altered to 1 July 1961, to allow her to become eligible for the matriculation exam. As a child, she was fascinated by aeroplanes and flying. She went to local flying clubs and watched planes with her father. After getting a Bachelor of Engine

In [29]:
%%time

import torch
from transformers import AlbertTokenizer, AlbertForQuestionAnswering
tokenizer = AlbertTokenizer.from_pretrained('ahotrod/albert_xxlargev1_squad2_512')
model = AlbertForQuestionAnswering.from_pretrained('D:/albert')

Wall time: 34.3 s


In [30]:
def answer(question, text):
    input_dict = tokenizer.encode_plus(question, text, return_tensors='pt')
    input_ids = input_dict["input_ids"].tolist()
    start_scores, end_scores = model(**input_dict,return_dict=False)
    
    ##Return_dict is important 
    start = torch.argmax(start_scores)
    end = torch.argmax(end_scores)
    
    #print("Scores are " ,start_scores ," ", end_scores,"\n")
    
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    answer = ''.join(all_tokens[start: end + 1]).replace('▁', ' ').strip()
    answer = answer.replace('[SEP]', '')
    return answer if answer != '[CLS]' and len(answer) != 0 else 'could not find an answer'

In [31]:
pattern3 = re.compile(r"[\n]+")
pattern4 = re.compile(r"[=]+") 


In [32]:
%%time
for i in range(len(Con_list)) :
    Con_list[i] = pattern3.sub("",Con_list[i])
    Con_list[i] = pattern4.sub("",Con_list[i])

Wall time: 0 ns


In [33]:
Con_list

['Kalpana Chawla (17 March 1962 – 1 February 2003) was an American astronaut, engineer, and the first woman of Indian origin to go to space. She first flew on Space Shuttle Columbia in 1997 as a mission specialist and primary robotic arm operator. In 2003, Chawla was one of the seven crew members who died in the Space Shuttle Columbia disaster when the spacecraft disintegrated during its re-entry into the Earth\'s atmosphere. Chawla was posthumously awarded the Congressional Space Medal of Honor, and several streets, universities, and institutions have been named in her honor. She is regarded as a national hero in India. Early life Kalpana Chawla was born on 17 March 1962, in Karnal of present-day Haryana, India, but her official date of birth was altered to 1 July 1961, to allow her to become eligible for the matriculation exam. As a child, she was fascinated by aeroplanes and flying. She went to local flying clubs and watched planes with her father. After getting a Bachelor of Engine

In [34]:
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [35]:
vector1 = text_to_vector(query) 

In [36]:
vector1

Counter({'What': 1, 'kalpana': 1, 'chawla': 1, 'spoke': 1})

In [37]:
len(Con_list)

1

##### Adding all text of Con_list 

In [38]:
text = ""
for i in range(len(Con_list)) :
    text += Con_list[i] 
    text += " "

In [39]:
text

'Kalpana Chawla (17 March 1962 – 1 February 2003) was an American astronaut, engineer, and the first woman of Indian origin to go to space. She first flew on Space Shuttle Columbia in 1997 as a mission specialist and primary robotic arm operator. In 2003, Chawla was one of the seven crew members who died in the Space Shuttle Columbia disaster when the spacecraft disintegrated during its re-entry into the Earth\'s atmosphere. Chawla was posthumously awarded the Congressional Space Medal of Honor, and several streets, universities, and institutions have been named in her honor. She is regarded as a national hero in India. Early life Kalpana Chawla was born on 17 March 1962, in Karnal of present-day Haryana, India, but her official date of birth was altered to 1 July 1961, to allow her to become eligible for the matriculation exam. As a child, she was fascinated by aeroplanes and flying. She went to local flying clubs and watched planes with her father. After getting a Bachelor of Enginee

In [40]:
text = text.split(".")

In [41]:
text

['Kalpana Chawla (17 March 1962 – 1 February 2003) was an American astronaut, engineer, and the first woman of Indian origin to go to space',
 ' She first flew on Space Shuttle Columbia in 1997 as a mission specialist and primary robotic arm operator',
 " In 2003, Chawla was one of the seven crew members who died in the Space Shuttle Columbia disaster when the spacecraft disintegrated during its re-entry into the Earth's atmosphere",
 ' Chawla was posthumously awarded the Congressional Space Medal of Honor, and several streets, universities, and institutions have been named in her honor',
 ' She is regarded as a national hero in India',
 ' Early life Kalpana Chawla was born on 17 March 1962, in Karnal of present-day Haryana, India, but her official date of birth was altered to 1 July 1961, to allow her to become eligible for the matriculation exam',
 ' As a child, she was fascinated by aeroplanes and flying',
 ' She went to local flying clubs and watched planes with her father',
 ' Aft

In [42]:
len(text)

78

In [43]:
%%time 

max_score = {}
for i in range(0,len(text)-1,1) :
    context = text[i] + text[i+1]
    vector2 = text_to_vector(context)
    max_score[get_cosine(vector1,vector2)] = i 

l = list(max_score.items())
l.sort(reverse=True)
max_score= dict(l)
    

Wall time: 3.05 ms


In [44]:
%%time
ans = "could not find an answer"
done = 0 
for k in max_score :
    index = max_score[k] 
    context = ""
    if(index != len(text)-1) :
        context = context = text[index] + text[index+1]
    else :
        context = context = text[index] + text[index-1]
    ans = answer(query,context)
    #print(ans,context)
    #print(done)
    done += 1 
    if(ans != "could not find an answer" ) :
        ans = ans.replace("[CLS]","")
        ans = ans.replace(query.lower(),"")
        
        print(ans) 
        print("And context is ....................\n")
        print(context)
        break 
    if(done == 20) :
        print("Sorry couldn't find a perfect answer !!!! :((")
        break 
        

"you are just your intelligence"
And context is ....................

 She spoke the following words while traveling in the weightlessness of space, "You are just your intelligence" On her first mission, Chawla traveled over 10
Wall time: 5.5 s


In [45]:
def finding_answer(Con_list,query) :
    vector1 = text_to_vector(query)
    pattern3 = re.compile(r"[\n]+")
    pattern4 = re.compile(r"[=]+") 
    
    for i in range(len(Con_list)) :
        Con_list[i] = pattern3.sub("",Con_list[i])
        Con_list[i] = pattern4.sub("",Con_list[i])
    
    text = ""
    for i in range(len(Con_list)) :
        text += Con_list[i] 
        text += " "
        
    text = text.split(".")
    
    
    max_score = {}
    for i in range(0,len(text)-1,1) :
        context = text[i] + text[i+1]
        vector2 = text_to_vector(context)
        max_score[get_cosine(vector1,vector2)] = i 

    l = list(max_score.items())
    l.sort(reverse=True)
    max_score= dict(l)
    
    
    ans = "could not find an answer"
    done = 0 
    for k in max_score :
        index = max_score[k] 
        context = ""
        if(index != len(text)-1) :
            context = context = text[index] + text[index+1]
        else :
            context = context = text[index] + text[index-1]
        ans = answer(query,context)
        #print(ans,context)
        #print(done)
        done += 1 
        if(ans != "could not find an answer" ) :
            ans = ans.replace("[CLS]","")
            ans = ans.replace(query.lower(),"")

            print(ans) 
            print("And context is ....................\n")
            print(context)
            break 
        if(done == 20) :
            print("Sorry couldn't find a perfect answer !!!! :((")
            break 
    
    
    
    
        
        
    
    
    

In [46]:
def complete(query) :
    query = query.lower()
    print("Query is ..............",query)
    Con_list , num_type = all_preprocessing(query)
    print()
    #print("Done preprocessing")
    #print(Con_list ,"is the content")
    if(num_type == 1) :
        answer = finding_answer(Con_list,query)
        print(answer)
    
    else :
        print(Con_list)
    

In [163]:
%%time 
complete("when MS Dhoni made his debut for India")

Query is .............. when ms dhoni made his debut for india
Searching for .......     ['ms', 'dhoni', 'made', 'debut', 'india'] 

Here is the list of similar words you may be looking for ...............

ms      ['mps', 'ms', 'cms', 'mes', 'rms'] 

dhoni      ['doi', 'don', 'donyi', 'dhvani', 'dhoni', 'dhondy', 'thondi', 'dhondo', 'dhani'] 

made      ['made', 'male', 'mandate', 'jade', 'aden', 'mahe', 'madhesi', 'remade', 'mande', 'mandare', 'maddale', 'mahadev', 'mandeep', 'maderna', 'mane'] 

debut      ['debut', 'but', 'deputy', 'debuts', 'debt', 'debutall', 'deb'] 

india      ['hindi', 'india', 'indian', 'indies', 'indiaall', 'india1933', 'indians', 'india1982', 'scindia', 'india1959', 'sindhi', 'nadia', 'indira', 'indonesia', 'india1950', 'india1932', 'india2013', 'india17th', 'india1919', 'india1907', 'india1952', 'india16th', 'india15th', 'india1879', 'india1984', 'gondia', 'india1937', 'india1956', 'india1898', 'india1977', 'india1973', 'india1925', 'indiaac', 'india1715',

### Sometimes the word you are searching for is not in the exact manner I've stores , so I can use the following code below written to find how I have stores that word in my database

In [159]:


for i in range(10088) :
    if "rao" in title[i].lower() :
        print(title[i],i)

Padmaja Rao 139
Simon Oraon 211
C. H. Hanumantha Rao 258
Saluri Rajeshwara Rao 301
Shamrao Madkaikar 405
Madhusudan Rao 487
P.V.Narasimha Rao 510
T. R. Subba Rao 578
Maya Rao 639
Duvvuri Subbarao 655
Baburao Govindrao Shirke 687
Malhar Rao Holkar 720
A. Ramana Rao 765
Ganpatrao Deshmukh 772
S. V. Ranga Rao 805
Rao Kadam Singh 836
Lalith J. Rao 848
K.L. Rao 849
Mysore Sadasiva Rao 960
Varavara Rao 1114
Muralidhar Rao 1310
Kartik Oraon 1632
V. Kasturi Ranga Varadarja Rao 1671
Ramoji Rao 1690
K. V. Krishna Rao 1811
Baburaoji Parkhe 1841
Rao Umrao Singh Bhati 2095
Rao Hashim Khan 2363
Dasari Narayana Rao 2489
Srirangam Srinivasarao 2506
Grandhi Mallikarjuna Rao 2535
Manikrao Thakre 2540
Kalvakuntla Chandrashekar Rao 2592
Narla Tata Rao 2704
A.N. Krishna Rao 2752
Pamulaparthi Sadasiva Rao 2882
A. S. Rao 2970
Aluru Venkata Rao 3131
Rao Tula Ram 3137
Ghantasala Venkateswara Rao 3146
Kaloji Narayana Rao 3174
B. Narsing Rao 3187
Joginder Rao 3214
U. R. Rao 3275
Burgula Ramakrishna Rao 3297
Kesh

In [107]:
"""
Problem with searches like A.P.J or C.V
"""

'\nProblem with searches like A.P.J or C.V\n'

In [49]:
%%time 

import spacy
nlp = spacy.load("en_core_web_sm")


Wall time: 3.11 s


In [50]:
%%time
doc = nlp("first test wickett of jasprit bumrah")

for tok in doc:
    print(tok, tok.pos_)

first PROPN
test PROPN
wickett PROPN
of ADP
jasprit PROPN
bumrah PROPN
Wall time: 115 ms


###### This births and deaths keyword can be used for counting primary keywords  

In [51]:
%%time
count =  0 
for i in range(10088) :
    for j in range(len(data_categories[i])) :
        if 'births' in data_categories[i][j]:
            count += 1
            break 
        if 'deaths' in data_categories[i][j] :
            count += 1
        
            
print(count)

6513
Wall time: 2.65 s


In [140]:
temp = ["a","b","c"]
temp.remove("d")

ValueError: list.remove(x): x not in list